In [11]:
import requests
import hashlib
import hmac
import time
import datetime
import pandas as pd
import numpy as np
import credentials
from urllib.parse import urljoin, urlencode

apikey = credentials.apikey
secret = credentials.secret

binance = 'https://api.binance.com'

# test = requests.get("https://api.binance.com/api/v1/ping")
# servertime = requests.get("https://api.binance.com/api/v1/time")

headers = {
    'X-MBX-APIKEY': apikey
}

# Get timestamp
def timestamp():
    timestamp = int(time.time() * 1000)
    
    return timestamp

def date_to_timestamp(date_string='01/01/2019'):
    date = datetime.datetime.strptime(date_string, "%d/%m/%Y")
    timestamp = datetime.datetime.timestamp(date)
    
    return timestamp

# Create signature when needed
def signature(timestamp, **params):
    querystring = urlencode(params)
    params['signature'] = hmac.new(secret.encode('utf-8'), querystring.encode('utf-8'), hashlib.sha256).hexdigest()
    
    return params

# Get price
def get_price(headers):
    path = '/api/v3/ticker/price'
    params = {'symbol': 'BTCUSDT'}
    r = requests.get(binance + path, headers=headers, params=params)
    
    return r.json()

print(get_price(headers))

{'symbol': 'BTCUSDT', 'price': '13061.25000000'}


In [12]:
# Get candles from 01/01/2010 to present

start = '2017'
end = '2020'
interval = '4h'
symbol = 'BTCUSDT'
start_time = int(date_to_timestamp(f'01/01/{start}') * 1000)

# Get candles function
def get_candles(symbol='BTCUSDT', limit=1000, interval='4h', startTime=None, endTime=None):
    path = '/api/v3/klines'
    params = {'symbol': symbol, 'limit': limit, 'interval': interval, 'startTime': startTime, 'endTime': endTime}
    
    r = requests.get(binance + path, params=params)
    
    return r.json()

# Function that gets the candles from a specific day until present and with an specific interval
def get_all_candles(startTime, interval='4h'):
    all_candles = []
    
    # Get the current timestamp
    current_timestamp = timestamp()
    
    # We add 2 min (120), if the current timestamp is higher than the stating time
    # It means that we are still retrieving all data, and if it is less it means that we are 
    # in the current date and hour.
    while current_timestamp > (startTime + 120):
        candles = get_candles(startTime=startTime, interval=interval)
        all_candles.append(candles)
        
        # update the startTime, we add 1 second to the time where the last candle ended.
        startTime = all_candles[-1][-1][6] + 1

    
    
    return all_candles

"""This function returns:
[
  [
    1499040000000,      // Open time
    "0.01634790",       // Open
    "0.80000000",       // High
    "0.01575800",       // Low
    "0.01577100",       // Close
    "148976.11427815",  // Volume
    1499644799999,      // Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "17928899.62484339" // Ignore.
  ]
]
"""
all_candles = get_all_candles(start_time, interval=interval)

In [13]:
def binance_fromtimestamp(binance_time):
    correct_date = datetime.datetime.fromtimestamp(binance_time/1000)
    
    return correct_date

In [14]:
# Add all the candles in one single dataframe
df = pd.DataFrame(all_candles[0])
for candles in all_candles[1:]:
    df_candles = pd.DataFrame(candles)
    df = pd.concat([df, df_candles], ignore_index=True)

# These are the column names from binance api data
columns=['open_time', 'open', 'high', 'low', 'close', 
         'volume', 'close_time', 'quote_asset_volume','number_of_trades', 
         'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

# Update the column names from the api data
df.columns = columns

# Change the timestamp values to real dates.
df['open_time'] = df.open_time.apply(binance_fromtimestamp)
df['close_time'] = df.close_time.apply(binance_fromtimestamp)

# Dropping irrelevant columns
df.drop(['ignore'], axis=1, inplace=True)

In [ ]:
# Save file
df.to_csv(f'bitcoin/btc_1m_{start}_{end}.csv')